This notebook trains the same VAE model multiple times and creates the artifacts needed for
https://mariogemoll.com/vae.

In [ ]:
# Clone the repo if needed (e.g. on Colab)
import os
import subprocess


def is_correct_repo() -> bool:
    try:
        result = subprocess.run(
            ["git", "remote", "get-url", "origin"], capture_output=True, text=True, check=True
        )
        remote_url = result.stdout.strip()
        return remote_url in [
            "https://github.com/mariogemoll/vae.git",
            "git@github.com:mariogemoll/vae.git",
        ]
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False


if not is_correct_repo():
    !git clone https://github.com/mariogemoll/vae.git

if not os.getcwd().endswith("vae/notebooks"):
    %cd vae/notebooks

In [ ]:
# Install Python dependencies
%pip install -r requirements-build.txt -q

In [ ]:
# Build UI widgets
subprocess.run(["npm", "i", "--no-progress"], cwd="../widgets", check=True)
subprocess.run(["npm", "i", "--no-progress"], cwd="widget-wrappers", check=True)
subprocess.run(["bash", "build_wrapped_widgets.sh"], cwd="widget-wrappers", check=True)

In [ ]:
import numpy as np
import torch
from tqdm.notebook import trange

from constants import hue_range, num_epochs, num_models, sidelength, size_range
from dataset import generate_dataset
from grid import make_standard_grid
from image import get_images
from model import VAE
from training import train
from util import compress_floats, expand_floats, get_device, onnx_export_to_files, stringify_coords
from vaewidgets import model_comparison

device = get_device()
if device.type == "cpu":
    print("Using CPU for training. You might want to switch to a GPU to speed things up!")

In [ ]:
trainset_coords, valset_coords, trainset, valset = generate_dataset(
    size_range=size_range,
    hue_range=hue_range,
    valset_size_range=(0.6, 0.9),
    valset_hue_range=(0.4, 0.7),
    num_samples=2000,
)

In [ ]:
trainset_coords_x, trainset_coords_y = stringify_coords(trainset_coords)
valset_coords_x, valset_coords_y = stringify_coords(valset_coords)
trainset_images_bytes = trainset.numpy().tobytes()
valset_images_bytes = valset.numpy().tobytes()
with open("trainset_coords.json", "w") as f_w:
    f_w.write(f'{{"x": [{trainset_coords_x}], "y": [{trainset_coords_y}]}}')
with open("valset_coords.json", "w") as f_w:
    f_w.write(f'{{"x": [{valset_coords_x}], "y": [{valset_coords_y}]}}')
with open("trainset_images.bin", "wb") as f_wb:
    f_wb.write(trainset_images_bytes)
with open("valset_images.bin", "wb") as f_wb:
    f_wb.write(valset_images_bytes)

In [ ]:
standard_grid = make_standard_grid(size_range, hue_range)
imgs = get_images(sidelength, [tuple(pair) for pair in standard_grid.reshape(-1, 2).tolist()])
grid_x = (torch.from_numpy(imgs).float() / 255.0).to(device)

In [ ]:
losses = []
processed_grids = []
vae = VAE(2).to(device)
batch_size_train = 256
batch_size_val = 64
for i in trange(num_models):
    filename = f"vae_{i}.pth"
    train_losses, val_losses, model_processed_grids = train(
        device,
        trainset,
        valset,
        filename,
        num_epochs,
        batch_size_train,
        batch_size_val,
        grid_x,
        True,
    )
    processed_grids.append(model_processed_grids)
    losses.append((train_losses, val_losses))

In [ ]:
loss_data = np.array(losses).astype(np.float32)
loss_data.tofile("losses.bin")
assert np.array_equal(
    loss_data, np.fromfile("losses.bin", dtype=np.float32).reshape(num_models, 2, num_epochs)
)

In [ ]:
all_processed_grids = np.array(processed_grids).reshape(num_models, num_epochs, 10, 10, 2)
with open("grids.bin", "wb") as f_wb:
    f_wb.write(compress_floats(all_processed_grids))
with open("grids.bin", "rb") as f_rb:
    grids_reconstructed = expand_floats(f_rb.read()).reshape(num_models, num_epochs, 10, 10, 2)
assert np.allclose(all_processed_grids, grids_reconstructed, rtol=1e-01, atol=1e-01)

In [ ]:
with open("losses.bin", "rb") as losses_f, open("grids.bin", "rb") as grids_f:
    losses_bytes = losses_f.read()
    grids_bytes = grids_f.read()

model_comparison(losses_bytes, grids_bytes)

In [ ]:
for i in range(9):
    vae = VAE(2)
    vae.load_state_dict(torch.load(f"vae_{i}.pth"))
    vae.eval()
    onnx_export_to_files(vae.encoder, vae.decoder, f"vae_{i}_encoder.onnx", f"vae_{i}_decoder.onnx")